# Import

In [1]:
import sys
import os
import datetime
import pandas as pd
sys.path.append("../")
sys.path.append("../..")

# data load
from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_Client_v2 as influx_Client, multipleDataSets

# split by holiday
from Clust.clust.transformation.splitDataByCondition import holiday

# setting common parameter : influxDB instance
db_client = influx_Client.influxClient(ins.CLUSTDataServer2)

- TimeSeries를 Holiday/notHoliday 기준으로 데이터를 나누는 것을 보여준다.
- 아래 순서에 따라 테스트가 진행
    1. Input : 단일 Dataframe
    2. Input : 특정 Bucket의 Dataset
- Holiday/notHoliday 를 나누는 모든 챕터들은 아래와 같은 진행 순서를 갖는다.
    1. data preparation
        - duration : 2022.02.01~2022.02.28
        - kWeather Indoor 초등학교 
    4. splitDataByCondition

# 1. Split data by holiday with dataframe

## 1-1. Data Preparation

In [2]:
## 1. Data Information
bucket_name ='air_indoor_초등학교'
measurement_name = "ICW0W2000025"

## 2. Time
start_time = pd.to_datetime("2022-02-01 00:00:00")
end_time = pd.to_datetime("2022-02-28 23:59:59")

In [3]:
data = db_client.get_data_by_time(start_time, end_time, bucket_name, measurement_name)

In [4]:
data

,in_co2,in_humi,in_noise,in_pm01,in_pm10,in_pm25,in_temp,in_voc
time,,,,,,,,
2022-02-01 00:00:00+00:00,397.0,20.0,47.0,43.0,53.0,46.0,15.5,7.0
2022-02-01 00:01:00+00:00,399.0,20.0,48.0,43.0,53.0,46.0,15.5,6.0
2022-02-01 00:02:00+00:00,407.0,20.0,48.0,43.0,53.0,46.0,15.5,6.0
2022-02-01 00:03:00+00:00,398.0,20.0,48.0,43.0,56.0,47.0,15.5,6.0
2022-02-01 00:04:00+00:00,401.0,20.0,48.0,43.0,55.0,46.0,15.5,7.0
...,...,...,...,...,...,...,...,...
2022-02-28 23:55:00+00:00,404.0,24.0,46.0,21.0,26.0,22.0,18.5,6.0
2022-02-28 23:56:00+00:00,403.0,24.0,46.0,21.0,26.0,22.0,18.5,5.0
2022-02-28 23:57:00+00:00,400.0,24.0,46.0,21.0,25.0,22.0,18.5,7.0


## 1-2. Split Data by holiday

In [8]:
split_data_result = holiday.get_split_data_by_holiday_from_dataframe(data)

In [9]:
split_data_result

{'holiday':                            in_co2  in_humi  in_noise  in_pm01  in_pm10  \
 time                                                                     
 2022-02-01 00:00:00+00:00   397.0     20.0      47.0     43.0     53.0   
 2022-02-01 00:01:00+00:00   399.0     20.0      48.0     43.0     53.0   
 2022-02-01 00:02:00+00:00   407.0     20.0      48.0     43.0     53.0   
 2022-02-01 00:03:00+00:00   398.0     20.0      48.0     43.0     56.0   
 2022-02-01 00:04:00+00:00   401.0     20.0      48.0     43.0     55.0   
 ...                           ...      ...       ...      ...      ...   
 2022-02-27 23:55:00+00:00   374.0     21.0      47.0      9.0     10.0   
 2022-02-27 23:56:00+00:00   374.0     21.0      46.0      9.0     10.0   
 2022-02-27 23:57:00+00:00   372.0     21.0      47.0      8.0      9.0   
 2022-02-27 23:58:00+00:00   375.0     21.0      47.0      8.0      9.0   
 2022-02-27 23:59:00+00:00   374.0     21.0      47.0      8.0     10.0   
 
            

In [10]:
split_data_result.keys()

dict_keys(['holiday', 'notholiday'])

# 2. Split data by holiday with Dataset

## 2-1. Data Preparation

In [11]:
## 1. Data Information
bucket_name ='air_indoor_초등학교'

## 2. Time
start_time = pd.to_datetime("2022-02-01 00:00:00")
end_time = pd.to_datetime("2022-02-28 23:59:59")

In [18]:
dataset = multipleDataSets.get_all_msdata_in_bucket(start_time, end_time, db_client, bucket_name)

## 2-2. Split Data by holiday

In [20]:
split_dataset_result = holiday.get_split_data_by_holiday_from_dataset(dataSet)

In [21]:
split_dataset_result

{'ICW0W2000022_elementarySchool': {'holiday':                            in_co2  in_humi  in_noise  in_pm01  in_pm10  \
  time                                                                     
  2022-02-01 00:00:00+00:00   471.0     26.0      48.0     29.0     33.0   
  2022-02-01 00:01:00+00:00   470.0     26.0      47.0     28.0     32.0   
  2022-02-01 00:02:00+00:00   471.0     26.0      47.0     28.0     31.0   
  2022-02-01 00:03:00+00:00   463.0     26.0      47.0     28.0     32.0   
  2022-02-01 00:04:00+00:00   468.0     26.0      48.0     28.0     33.0   
  ...                           ...      ...       ...      ...      ...   
  2022-02-27 23:55:00+00:00   455.0     25.0      47.0      5.0      6.0   
  2022-02-27 23:56:00+00:00   451.0     25.0      47.0      5.0      6.0   
  2022-02-27 23:57:00+00:00   457.0     25.0      47.0      5.0      6.0   
  2022-02-27 23:58:00+00:00   464.0     25.0      47.0      5.0      6.0   
  2022-02-27 23:59:00+00:00   459.0     25.0

In [22]:
split_dataset_result.keys()

dict_keys(['ICW0W2000022_elementarySchool', 'ICW0W2000023_elementarySchool', 'ICW0W2000024_elementarySchool', 'ICW0W2000025_elementarySchool', 'ICW0W2000030_elementarySchool', 'ICW0W2000031_elementarySchool', 'ICW0W2000032_elementarySchool', 'ICW0W2000033_elementarySchool', 'ICW0W2000034_elementarySchool'])

In [23]:
split_dataset_result["ICW0W2000022_elementarySchool"]

{'holiday':                            in_co2  in_humi  in_noise  in_pm01  in_pm10  \
 time                                                                     
 2022-02-01 00:00:00+00:00   471.0     26.0      48.0     29.0     33.0   
 2022-02-01 00:01:00+00:00   470.0     26.0      47.0     28.0     32.0   
 2022-02-01 00:02:00+00:00   471.0     26.0      47.0     28.0     31.0   
 2022-02-01 00:03:00+00:00   463.0     26.0      47.0     28.0     32.0   
 2022-02-01 00:04:00+00:00   468.0     26.0      48.0     28.0     33.0   
 ...                           ...      ...       ...      ...      ...   
 2022-02-27 23:55:00+00:00   455.0     25.0      47.0      5.0      6.0   
 2022-02-27 23:56:00+00:00   451.0     25.0      47.0      5.0      6.0   
 2022-02-27 23:57:00+00:00   457.0     25.0      47.0      5.0      6.0   
 2022-02-27 23:58:00+00:00   464.0     25.0      47.0      5.0      6.0   
 2022-02-27 23:59:00+00:00   459.0     25.0      47.0      5.0      6.0   
 
            